# Tornadoes [1950-2022]

Source: https://www.kaggle.com/datasets/sujaykapadnis/tornados



### Questions:
1. **Tornado severity**
- Has the length of paths changed? (look at both the average, and median)
- Has the severity (on F/EF scale) changed?

2. **Tornado freqency**
- Has tornado frequency increased over time?
- Has the freqency of more severe tornados increased over time?

3. **Tornado zone** (Optional)
- Has "tornado alley" increased/decreased in size over time? (look at "states effected")
- And/or has this area moved?  



### Data Acquisition:  
Here are the first few rows of the file, `Tornadoes.csv`.

In [ ]:
import pandas as pd

tornadoes = pd.read_csv('tornados.csv', index_col=None)
tornadoes.head()

In [ ]:
tornadoes.info()

In [ ]:
tornadoes.head()

Here's a brief overview of the columns:  

- `om`: Integer - Tornado number. Effectively an ID for this tornado in this year.
- `yr`: Integer - Year, 1950-2022.
- `mo`: Integer - Month, 1-12.
- `dy`: Integer - Day of the month, 1-31.
- `date`: Date - Date.
- `time`: Time - Time.
- `tz`: Character - Canonical tz database timezone.
- `datetime_utc`: Datetime - Date and time normalized to UTC.
- `st`: Character - Two-letter postal abbreviation for the state (DC = Washington, DC; PR = Puerto Rico; VI = Virgin Islands).
- `stf`: Integer - State FIPS (Federal Information Processing Standards) number.
- `mag`: Integer - Magnitude on the F scale (EF beginning in 2007). Some of these values are estimated (see fc).
- `inj`: Integer - Number of injuries. When summing for state totals, use sn == 1 (see below).
- `fat`: Integer - Number of fatalities. When summing for state totals, use sn == 1 (see below).
- `loss`: Double - Estimated property loss information in dollars. Prior to 1996, values were grouped into ranges. The reported number for such years is the maximum of its range.
- `slat`: Double - Starting latitude in decimal degrees.
- `slon`: Double - Starting longitude in decimal degrees.
- `elat`: Double - Ending latitude in decimal degrees.
- `elon`: Double - Ending longitude in decimal degrees.
- `len`: Double - Length in miles.
- `wid`: Double - Width in yards.
- `ns`: Integer - Number of states affected by this tornado. 1, 2, or 3.
- `sn`: Integer - State number for this row. 1 means the row contains the entire track information for this state, 0 means there is at least one more entry for this state for this tornado (om + yr).
- `f1`: Integer - FIPS code for the 1st county.
- `f2`: Integer - FIPS code for the 2nd county.
- `f3`: Integer - FIPS code for the 3rd county.
- `f4`: Integer - FIPS code for the 4th county.
- `fc`: Logical - Was the mag column estimated?

### Initial Data Exploration

Let's begin by:
1. Checking the shape (number of rows and columns) of the dataset. 
2. Identifying the data type for each variable.  
3. Obtaining basic statistics for quantitative/numeric columns.  


In [ ]:
# Get the size of the dataset
print("There are " + str(tornadoes.shape[1]) + " columns and " + str(tornadoes.shape[0]) + " rows. ")

# Get data types for each column
print("\nThese are the data types:\n", tornadoes.dtypes)

# Get basic statistics for numeric columns
print("\nThese are the basic statistics for the numeric columns:\n", tornadoes.describe())


Here's a summary of our exploration:

### Dataset Size: 
- Number of rows (tornadoes): 68,693
- Number of columns (variables): 21

### Data Types:

- **Numerical Data (int64 or float64)**: `om`, `yr`, `mo`, `dy`, `stf`, `mag`, `inj`, `fat`, `loss`, `slat`, `slon`, `elat`, `elon`, `len`, `wid`, `ns`, `sn`, `f1`, `f2`, `f3`, `f4`
- **Categorical Data (object)**: `date`, `time`, `tz`, `datetime_utc`, `st`
- **Boolean Data (bool)**: `fc`

### Basic Statistics:
For the numerical columns:
- `om`: Ranges from 1 to 622,080.
- `yr`: Ranges from 1950 to 2022.
- `mo`: Ranges from 1 to 12.
- `dy`: Ranges from 1 to 31.
- `stf`: Ranges from 1 to 78.
- `mag`: Ranges from 0 to 5.
- `inj`: Ranges from 0 to 1,740.
- `fat`: Ranges from 0 to 158.
- `loss`: Ranges from $50 to $2,800,100,000.
- `slat`: Ranges from 17.7212 to 61.02.
- `slon`: No specific range provided.
- `elat`: Has a minimum value of 0.
- `elon`: No specific range provided.
- `len`: Ranges from 0 to 234.7 miles.
- `wid`: Ranges from 0 to 4,576 yards.
- `ns`: Ranges from 1 to 3.
- `sn`: Ranges from 0 to 1.
- `f1`: Has a maximum value of 810.
- `f2`: Has a maximum value of 820.
- `f3`: Has a maximum value of 710.
- `f4`: Has a maximum value of 507.



Next, we'll proceed with the Data Cleansing phase. 

### Data Cleansing: 
1. **Handle missing values**: 
- We'll first identify columns with missing values.  
- Then we'll decide whether the impute, fill, or drop these missing values based on the nature and significance of each column.  

2. **Remove duplicates**:
- We'll check for duplicate rows and remove them if any.  

3. **Data type conversion**:
- Ensure that each column is of the correct data type.  

4. **Outlier detection**:
- We'll briefly look for potential outliers using statistical methods or visualization.  

5. **Standardization**:
- If necessary, we'll standardize values. This step might be more relevant during modeling, especially if we use algorithms sensitive to feature scales. 

Let's start by checking for missing values. 


In [ ]:
# Check for missing values
missing_values = tornadoes.isnull().sum()

# Calculate missing value counts
missing_values = pd.DataFrame(missing_values[missing_values > 0])

# Calculate missing percent of values for each column
missing_pct = ((missing_values/tornadoes.shape[0]*100).round(3))

# Add missing count and percent to table
missing_values["% Missing"] = missing_pct
missing_values.rename(columns={0: 'Count Missing'}, inplace=True)
missing_values

Here are the columns with missing values:

- **mag**: 756 missing values, or 1.101%
- **loss**: 27170 missing values, or 38.553%

Let's strategize how to handle these:
- **mag**: Drop rows with missing values since it only accounts for about 1% of data.
- **loss**: Dropping this variable since it isn't needed.

In [ ]:
# Drop NA values from mag.
tornadoes.dropna(subset=['mag'], inplace=True)

# Drop loss variable.
tornadoes.drop(['loss'], axis=1, inplace=True)

In [ ]:
tornadoes.info()

Now that missing values have been handled, we'll:
2. **Remove Duplicates**
3. **Convert Datatypes**
4. **Detect Outliers**

Let's start by checkinf for duplicate rows.  

In [ ]:
# Check for duplicate rows
duplicate_rows = tornadoes.duplicated().sum()
print("There are " + str(duplicate_rows) + " duplicate row(s).\n")


# Get a boolean series indicating which rows are duplicates (including the original rows)
duplicate_mask_all = tornadoes.duplicated(keep=False)

# Use this mask to filter and display both the original and duplicate rows
duplicate_rows_all_df = tornadoes[duplicate_mask_all]
duplicate_rows_all_df

In [ ]:
# Remove duplicate rows, if any
if duplicate_rows > 0: 
    tornadoes.drop_duplicates(inplace=True)

# Verify that the removal worked
duplicate_rows = tornadoes.duplicated().sum()
print("There are " + str(duplicate_rows) + " duplicate row(s).\n")

Next, let's check the data types in each column, and convert if necessary.  

In [ ]:
# Display data types for each column
current_data_types = tornadoes.dtypes
current_data_types

These data types seem app
**Numerical Data Types**:
- Continuous variables (floats): 
- Discrete variables (ints): `yr`, 
- Date/time variables (DateTime): `date`, `time`

**Categorical Data Types**:
- Dummy/indicator/coded variables (ints): `om`, `mo`, `dy`, 
- Strings/text information: 

The current data types for each column are as follows:

### Numerical Data Types
**Continuous variables (floats)**: 
- `loss`: Estimated property loss in dollars.
- `slat`: Starting latitude in decimal degrees.
- `slon`: Starting longitude in decimal degrees.
- `elat`: Ending latitude in decimal degrees.
- `elon`: Ending longitude in decimal degrees.
- `len`: Length in miles.
- `wid`: Width in yards.

**Discrete variables (ints)**: 
- `yr`: Year, 1950-2022.
- `stf`: State FIPS number.
- `mag`: Magnitude on the F scale.
- `inj`: Number of injuries.
- `fat`: Number of fatalities.
- `ns`: Number of states affected by this tornado.
- `sn`: State number for this row.
- `f1`: FIPS code for the 1st county.
- `f2`: FIPS code for the 2nd county.
- `f3`: FIPS code for the 3rd county.
- `f4`: FIPS code for the 4th county.

**Date/time variables (DateTime)**: 
- `date`: Date.
- `time`: Time.
- `datetime_utc`: Date and time normalized to UTC.


### Categorical Data Types
**Dummy/indicator/coded variables (ints)**: 
- `om`: Tornado number (ID).
- `mo`: Month, 1-12.
- `dy`: Day of the month, 1-31.
- `stf`: State FIPS number.
- `mag`: Magnitude on the F scale.
- `ns`: Number of states affected by this tornado.
- `sn`: State number for this row.
- `f1`: FIPS code for the 1st county.
- `f2`: FIPS code for the 2nd county.
- `f3`: FIPS code for the 3rd county.
- `f4`: FIPS code for the 4th county.

**Strings/textual information**: 
- `tz`: Canonical tz database timezone.
- `st`: Two-letter postal abbreviation for the state.

**Logical/Binary (usually int or bool)**: 
- `fc`: Was the `mag` column estimated?
